# Import 

In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import PIL
from pathlib import Path
import random
import glob
import shutil
import os
import cv2


In [3]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
IMAGE_SIZE = [176, 208]
EPOCHS = 100

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Downsampling Dataset

Saving path of orginal dataset directory and downsampled dataset directory

In [5]:
path_origin_ds = r"/content/drive/MyDrive/Colab Notebooks/Alzheimer_s Dataset/"
path_dwn_samp_ds = r"/content/drive/MyDrive/Colab Notebooks/Downsampled_dataset/"

Copying the orginal dataset into the downsampled dataset directory


In [7]:
shutil.copytree(path_origin_ds, path_dwn_samp_ds)

'/content/drive/MyDrive/Colab Notebooks/Downsampled_dataset/'

Check the number of images in each directory to determine which are the majority classes

In [9]:
for dirname, _, _ in os.walk(path_dwn_samp_ds+"train/"):
    if dirname != path_dwn_samp_ds+"train/":
        print('     ',len(os.listdir(dirname)), 'images in', dirname)

      1792 images in /content/drive/MyDrive/Colab Notebooks/Downsampled_dataset/train/VeryMildDemented
      52 images in /content/drive/MyDrive/Colab Notebooks/Downsampled_dataset/train/ModerateDemented
      717 images in /content/drive/MyDrive/Colab Notebooks/Downsampled_dataset/train/MildDemented
      2560 images in /content/drive/MyDrive/Colab Notebooks/Downsampled_dataset/train/NonDemented


Saving the directory path of each class of images

In [7]:
path_VMDem = path_dwn_samp_ds+"train/VeryMildDemented/"
path_ModDem = path_dwn_samp_ds+"train/ModerateDemented/"
path_MilDem = path_dwn_samp_ds+"train/MildDemented/"
path_NonDem = path_dwn_samp_ds+"train/NonDemented/"

We reduce (by selecting randomly the images to remove) the number of images of the VeryMildDemented class and the NonDemented class to reach the number of images of the MildDemented class

In [8]:
def delete_images(directory, number_of_images):
    images = os.listdir(directory)
    for image in random.sample(images, number_of_images):
        path_file = directory + image
        os.remove(path_file)

goal_nb_im = len(os.listdir(path_MilDem))
nb_VMDem_to_delete = -(goal_nb_im - len(os.listdir(path_VMDem)))
nb_NonDem_to_delete = -(goal_nb_im - len(os.listdir(path_NonDem)))

#delete_images(path_VMDem, nb_VMDem_to_delete)
#delete_images(path_NonDem,nb_NonDem_to_delete)


Check that the images are correctly removed

In [13]:
for dirname, _, _ in os.walk(path_dwn_samp_ds+"train/"):
    if dirname != path_dwn_samp_ds+"train/":
        print('     ',len(os.listdir(dirname)), 'images in', dirname)

      717 images in /content/drive/MyDrive/Colab Notebooks/Downsampled_dataset/train/VeryMildDemented
      52 images in /content/drive/MyDrive/Colab Notebooks/Downsampled_dataset/train/ModerateDemented
      717 images in /content/drive/MyDrive/Colab Notebooks/Downsampled_dataset/train/MildDemented
      717 images in /content/drive/MyDrive/Colab Notebooks/Downsampled_dataset/train/NonDemented


# Oversampling the dataset

In [10]:
path_ovr_samp_ds = r"/content/drive/MyDrive/Colab Notebooks/Oversampled_dataset/"

In [18]:
shutil.copytree(path_origin_ds, path_ovr_samp_ds)

'/content/drive/MyDrive/Colab Notebooks/Oversampled_dataset/'

In [19]:
for dirname, _, _ in os.walk(path_ovr_samp_ds+"train/"):
    if dirname != path_ovr_samp_ds+"train/":
        print('     ',len(os.listdir(dirname)), 'images in', dirname)

      1792 images in /content/drive/MyDrive/Colab Notebooks/Oversampled_dataset/train/VeryMildDemented
      52 images in /content/drive/MyDrive/Colab Notebooks/Oversampled_dataset/train/ModerateDemented
      717 images in /content/drive/MyDrive/Colab Notebooks/Oversampled_dataset/train/MildDemented
      2560 images in /content/drive/MyDrive/Colab Notebooks/Oversampled_dataset/train/NonDemented


In [20]:
path_VMDem = path_ovr_samp_ds+"train/VeryMildDemented/"
path_ModDem = path_ovr_samp_ds+"train/ModerateDemented/"
path_MilDem = path_ovr_samp_ds+"train/MildDemented/"
path_NonDem = path_ovr_samp_ds+"train/NonDemented/"

Define how we augment the images

In [97]:

def augment(img_pth):
  im = cv2.imread(img_pth)
  im = tf.image.random_flip_left_right(im)
  r = random.randint(1,4)
  im = tf.image.rot90(im,k=r)
  im = tf.image.random_brightness(im, max_delta=0.1)
  return im


We augment the images of the MildDemented class by calling the following cell once to reach more ore less 1500 images in this class

In [114]:

list_im = os.listdir(path_MilDem)

for img in list_im:
  im_path = path_MilDem+img
  augmented_im = augment(im_path)
  cv2.imwrite(path_MilDem + "(augm)" + img, augmented_im.numpy())


We augment the images of the ModerateDemented class by calling the following cell log2(1500/50) times until we reach more or less 1500 images for this class

In [129]:
list_im = os.listdir(path_ModDem)
print(len(list_im))
for img in list_im:
  im_path = path_ModDem + img
  augmented_im = augment(im_path)
  rdm = random.randint(1,20)
  cv2.imwrite(path_ModDem + "(augm)" + str(rdm) + img, augmented_im.numpy())

1218


We downsample the number of images of the majority classes to reach more or less 1500 images

In [131]:
delete_images(path_VMDem, 200)
delete_images(path_NonDem,1000)

We check the number of images for the Oversampled dataset

In [11]:
for dirname, _, _ in os.walk(path_ovr_samp_ds+"train/"):
    if dirname != path_ovr_samp_ds+"train/":
        print('     ',len(os.listdir(dirname)), 'images in', dirname)

      1592 images in /content/drive/MyDrive/Colab Notebooks/Oversampled_dataset/train/VeryMildDemented
      1576 images in /content/drive/MyDrive/Colab Notebooks/Oversampled_dataset/train/ModerateDemented
      1560 images in /content/drive/MyDrive/Colab Notebooks/Oversampled_dataset/train/NonDemented
      1438 images in /content/drive/MyDrive/Colab Notebooks/Oversampled_dataset/train/MildDemented
